# LlamaIndex의 사전 준비

In [1]:
# 패키지 설치
!pip install llama-index==0.6.12

In [2]:
# 환경 변수 준비
import os
os.environ["OPENAI_API_KEY"] = "<OpenAI_API의_API_키>"

In [3]:
import logging
import sys

# 로그 레벨 설정
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, force=True)

# Faiss를 이용한 질의응답

In [4]:
# 패키지 설치
#!pip install faiss-gpu

faiss 설치가 잘 안될 경우 다음 문서를 참조합니다. - 옮긴이  
https://github.com/facebookresearch/faiss/blob/main/INSTALL.md

In [5]:
from llama_index import SimpleDirectoryReader

# 문서 불러오기 (data 폴더에 문서를 넣어 두세요)
documents = SimpleDirectoryReader("data").load_data()

INFO:numexpr.utils:Note: NumExpr detected 20 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
DEBUG:llama_index.readers.file.base:> [SimpleDirectoryReader] Total files added: 7


In [6]:
import faiss

# faiss의 인덱스 생성
faiss_index = faiss.IndexFlatL2(1536)

INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.


In [7]:
from llama_index import GPTVectorStoreIndex, StorageContext
from llama_index.vector_stores.faiss import FaissVectorStore

# 인덱스 생성
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = GPTVectorStoreIndex.from_documents(
    documents, 
    storage_context=storage_context
)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): openaipublic.blob.core.windows.net:443
DEBUG:urllib3.connectionpool:https://openaipublic.blob.core.windows.net:443 "GET /gpt-2/encodings/main/vocab.bpe HTTP/1.1" 200 456318
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): openaipublic.blob.core.windows.net:443
DEBUG:urllib3.connectionpool:https://openaipublic.blob.core.windows.net:443 "GET /gpt-2/encodings/main/encoder.json HTTP/1.1" 200 1042301
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: 제1장: 데이터 프론트

밤이 되면 반짝이는 네오 도쿄. 고층 빌딩이 늘어서고, 네온...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: 제2장: 울프 코퍼레이션의 함정

미코는 목적지인 술집 '할머니의 집'으로 향하는 길...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: 제3장: 배신과 재회

술집 '할머니의 집'에서 미코는 데이터를 받을 사람인 료를 기...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: 제4장: 울프 코퍼레이션의 붕괴

미코와 료는 해커 집단과 함께 울프 코퍼레이션에 대...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: 제5장: 결전의 순간

미코와 료는 마침내 울프 코퍼레이션의

DEBUG:urllib3.util.retry:Converted retries value: 2 -> Retry(total=2, connect=None, read=None, redirect=None, status=None)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.openai.com:443
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/embeddings HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=68 request_id=b10707a3d8abaa90576d75c8330cc546 response_code=200
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 2888 tokens


In [8]:
# 쿼리 엔진 생성
query_engine = index.as_query_engine()

In [9]:
# 질의응답
print(query_engine.query("미코의 소꿉친구 이름은?"))

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\ubbf8\\ucf54\\uc758 \\uc18c\\uafc9\\uce5c\\uad6c \\uc774\\ub984\\uc740?"], "model": "text-embedding-ada-002", "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/embeddings HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=31 request_id=97584c9c3df4712c085dd334a96c7ed9 response_code=200
DEBUG:llama_index.indices.utils:> Top 2 nodes:
> [Node 4] [Similarity score:             0.336006] 제5장: 결전의 순간

미코와 료는 마침내 울프 코퍼레이션의 최상층에 도착해 CEO인 교활한 울프 박사와 대면한다. 울프 박사는 시민을 지배하려는 사악한 야망을 드러내며 자신...
> [Node 2] [Similarity score:             0.343525] 제3장: 배신과 재회

술집 '할머니의 집'에서 미코는 데이터를 받을 사람인 료를 기다리고 있었다. 료는 그녀의 어릴 적 친구이자 그 역시 울프 코퍼레이션과 싸우는 해커 집단...
INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token u

# Pinecone을 활용한 질의응답

In [10]:
# 패키지 설치
!pip install pinecone-client
!pip install transformers

                                              0.0/179.1 kB ? eta -:--:--
     -------------------------------------- 179.1/179.1 kB 5.4 MB/s eta 0:00:00
                                              0.0/60.0 kB ? eta -:--:--
     ---------------------------------------- 60.0/60.0 kB 3.3 MB/s eta 0:00:00
                                              0.0/300.0 kB ? eta -:--:--
     ------------------------------------- 300.0/300.0 kB 19.3 MB/s eta 0:00:00
                                              0.0/74.5 kB ? eta -:--:--
     ---------------------------------------- 74.5/74.5 kB ? eta 0:00:00
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
                                              0.0/7.4 MB ? eta -:--:--
     --                                       0.5/7.4 MB 10.9 MB/s eta 0:00:01
     --------                                 1.5/7.4 MB 16.0 MB/s eta 0:00:01
     ---------------                          2.9/7.4 MB 20.2 MB/s eta 0:00:01
     -----------------------          

In [11]:
from llama_index import SimpleDirectoryReader

# 문서 불러오기 (data 폴더에 문서를 넣어 두세요)
documents = SimpleDirectoryReader("data").load_data()

DEBUG:llama_index.readers.file.base:> [SimpleDirectoryReader] Total files added: 7


In [12]:
import pinecone

# pinecone-client의 인덱스 생성
api_key = "<Pinecone API 키>"
pinecone.init(api_key=api_key, environment="asia-southeast1-gcp-free")
pinecone.create_index(
    "quickstart",
    dimension=1536,
    metric="dotproduct",
    pod_type="p1"
)
pinecone_index = pinecone.Index("quickstart")

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): controller.us-west1-gcp.pinecone.io:443
DEBUG:urllib3.connectionpool:https://controller.us-west1-gcp.pinecone.io:443 "GET /actions/whoami HTTP/1.1" 401 114


In [13]:
from llama_index import GPTVectorStoreIndex, StorageContext
from llama_index.vector_stores.pinecone import PineconeVectorStore

# 인덱스 생성
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = GPTVectorStoreIndex.from_documents(
    documents, 
    storage_context=storage_context
)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


DEBUG:llama_index.node_parser.node_utils:> Adding chunk: 제1장: 데이터 프론트

밤이 되면 반짝이는 네오 도쿄. 고층 빌딩이 늘어서고, 네온...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: 제2장: 울프 코퍼레이션의 함정

미코는 목적지인 술집 '할머니의 집'으로 향하는 길...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: 제3장: 배신과 재회

술집 '할머니의 집'에서 미코는 데이터를 받을 사람인 료를 기...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: 제4장: 울프 코퍼레이션의 붕괴

미코와 료는 해커 집단과 함께 울프 코퍼레이션에 대...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: 제5장: 결전의 순간

미코와 료는 마침내 울프 코퍼레이션의 최상층에 도착해 CEO인...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: 제6장: 진실의 해방

미코는 울프 박사의 약점을 파고들어 그를 쓰러뜨리는데 성공한다...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: 제7장: 새로운 시작

울프 코퍼레이션이 무너진 후, 미코와 료는 서로의 과거를 용서...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\uc81c1\\uc7a5: \\ub370\\uc774\\ud130 \\ud504\\ub860\\ud2b8  \\ubc24\\uc774 \\ub418\\uba74 \\ubc

DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=36 request_id=36047b4ea6eb0dfb0c1c795d08ff4827 response_code=200
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 2888 tokens


In [14]:
# 쿼리 엔진 생성
query_engine = index.as_query_engine()

In [15]:
# 질의응답
print(query_engine.query("미코의 소꿉친구 이름은?"))

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\ubbf8\\ucf54\\uc758 \\uc18c\\uafc9\\uce5c\\uad6c \\uc774\\ub984\\uc740?"], "model": "text-embedding-ada-002", "encoding_format": "base64"}' message='Post details'
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=318 request_id=dbff070d27a230ec298417180f6b8d6e response_code=200
DEBUG:llama_index.indices.utils:> Top 2 nodes:
> [Node a193cf2b-be53-438e-b121-b57eded478bd] [Similarity score:             0.831997] 제5장: 결전의 순간

미코와 료는 마침내 울프 코퍼레이션의 최상층에 도착해 CEO인 교활한 울프 박사와 대면한다. 울프 박사는 시민을 지배하려는 사악한 야망을 드러내며 자신...
> [Node 58cbd934-4340-4927-bcb3-654e7c8c0325] [Similarity score:             0.828237] 제3장: 배신과 재회

술집 '할머니의 집'에서 미코는 데이터를 받을 사람인 료를 기다리고 있었다. 료는 그녀의 어릴 적 친구이자 그 역시 울프 코퍼레이션과 싸우는 해커 집단...
INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
INFO:llama